In [1]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

<ipython-input-1-863b54eaad45>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
df = pd.read_csv(r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\bike_share_locations.csv")
df.head()

,RouteID,BikeID,Location,Latitude,Longitude,OriginDestination
0,14193596,19593,Knight Law Center,44.043053,-123.069570,Origin
1,14193597,19710,Knight Law Center,44.043103,-123.069593,Origin
2,14193598,19734,Eugene Train Station,44.055235,-123.092733,Origin
3,14193600,19617,NaN,44.044918,-123.085150,Origin
4,14193601,19779,Erb Memorial Union overflow,44.045250,-123.072697,Origin


In [4]:
df["geom"] = df["Latitude"].map(str) + ',' + df['Longitude'].map(str)
df["geom"][0]

'44.0430533333333,-123.06957'

In [5]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [6]:
tqdm_notebook.pandas()

In [ ]:
df['address'] = df['geom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/8886 [00:00<?, ?it/s]

In [ ]:
df.address[1]